## Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import re
import ast
from datasets import load_dataset
from tqdm import tqdm
import helpers
import soft_prompting
import importlib
from transformers import AutoModelForCausalLM, AutoTokenizer
importlib.reload(helpers)
importlib.reload(soft_prompting)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# CONSTANTS
NUM_EPOCHS_FT = 100
NUM_EPOCHS_KD = 100
BATCH_SIZE = 32

## Get Data and preprocess it

In [ ]:
### train data
# data with label and image data
df_train_label = pd.DataFrame(load_dataset('derek-thomas/ScienceQA', split='train'))
df_train_label = df_train_label[df_train_label['solution'] != ''].reset_index()
df_train_label['input'] = df_train_label.apply(lambda row: helpers.build_prompt(row)[0], axis=1)
df_train_label[['index', 'input', 'answer', 'explanation', 'image']] = df_train_label[['index', 'input', 'answer', 'solution', 'image']]

# data from Gemini for KD
df_train_gemini = pd.read_csv('gemini_1_5_flash_output_train.csv', sep="\t")[['index', 'input', 'answer', 'explanation']]
df_train_gemini = pd.merge(df_train_gemini, df_train_label[['index', 'image']], on='index')

In [ ]:
### val data
df_val = pd.DataFrame(load_dataset('derek-thomas/ScienceQA', split='validation'))

In [ ]:
### Build Dataset class
class SoftPromptingDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.data = self.__preprocess_data(df)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample

    def __preprocess_data(self, data):
        data['input'] = data.apply(lambda row: self.tokenizer.encode(row['input'], return_tensors='pt'), axis=1)
        return data

## Functions for model training

In [ ]:
def train(dataloader_train, dataloader_val):
    train_errors = []
    val_errors = []
    for epoch in tqdm(range(NUM_EPOCHS_FT)):
        error = 0
        num_samples = 0
        for input, answer, explanation, image in dataloader_train:
            pass
            # forward
            # loss = ...
            # loss.backward()
            # optimizer.step()
            # error += loss.item() * X.size(0)
            # num_samples += X.size(0)
        error /= num_samples
        print(f'Error after epoch {epoch}: {error}')
        train_errors.append((epoch, error))
        if epoch % 10:
            val_error = 0
            num_samples
            for input, answer, explanation, image in dataloader_val:
                pass
                # forward
                # loss = ...
                # val_error += loss.item() * X.size(0)
                # num_samples += X.size(0)
            val_error /= num_samples
            print(f'Validation error after epoch {epoch}: {val_error}')
            val_errors.append((epoch, val_error))
    return train_errors_ft, val_error

In [ ]:
def visualize_error(train_errors, val_errors):
    plt.plot(zip(*train_errors), label="Train Error", marker="o", linestyle="-")
    plt.plot(zip(*val_errors), label="Train Error", marker="o", linestyle="-")
    plt.title("Train and Validation Error over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.show()

## Soft prompting using labels

In [ ]:
model_name = "Qwen/Qwen2.5-Coder-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_fine_tuned = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)
# DataLoader for train data
dataset_label_train = SoftPromptingDataset(df_train_label, tokenizer)
dataloader_label_train=DataLoader(dataset_label_train, batch_size=32, shuffle=True)
# DataLoader for val data
dataset_label_val = SoftPromptingDataset(df_val, model_fine_tuned)
dataloader_label_val=DataLoader(dataset_label_val, batch_size=32, shuffle=True)

In [ ]:
train_errors_ft, val_errors_ft = train(dataloader_label_train, dataloader_label_val)

In [ ]:
visualize_error(train_errors_ft, val_errors_ft)

## Knowledge Distillation

In [ ]:
model = None # ToDo
model_knowledge_distillation = soft_prompting.MultimodalSoftPrompting.from_pretrained(model)
# DataLoader for train data
dataset_gemini_train = SoftPromptingDataset(df_train_gemini, model_fine_tuned)
dataloader_gemini_train=DataLoader(dataset_gemini_train, batch_size=32, shuffle=True)
# DataLoader for val data
dataset_gemini_val = SoftPromptingDataset(df_val, model_fine_tuned)
dataloader_gemini_val=DataLoader(dataset_gemini_val, batch_size=32, shuffle=True)

In [ ]:
train_errors_kd, val_errors_kd = train(dataset_gemini_train, dataloader_gemini_train)

In [ ]:
visualize_error(train_errors_kd, val_errors_kd)